In [5]:
import pandas as pd
import numpy as np
from gurobipy import Model, GRB

# Load data
data = pd.read_csv("/Users/yashshah/Downloads/Telecomunication+Antenna.csv", index_col=0)
locations = data.index.tolist()
x = data['X coordinate'].values
y = data['Y coordinate'].values
pop = data['Population'].values

# Compute Euclidean distance matrix
def compute_distance_matrix(x_coords, y_coords, locations):
    n = len(locations)
    dist_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            dist = np.sqrt((x_coords[j] - x_coords[i])**2 + (y_coords[j] - y_coords[i])**2)
            dist_matrix[i, j] = dist
    return pd.DataFrame(dist_matrix, index=locations, columns=locations)

distance_df = compute_distance_matrix(x, y, locations)

# Coverage matrices
def calculate_coverage_matrices(distance_df):
    coverage_basic = (distance_df <= 4.5).astype(int)
    coverage_extended = (distance_df <= 8.0).astype(int)
    return coverage_extended, coverage_basic

coverage_ext, coverage_bas = calculate_coverage_matrices(distance_df)

# Optimization model
model = Model("Antenna_Placement")

n = len(locations)

# Decision variables: 1 if basic/ext antenna is placed at location i
basic = model.addVars(n, vtype=GRB.BINARY, name="basic")
ext = model.addVars(n, vtype=GRB.BINARY, name="extended")

# Covered or not
covered = model.addVars(n, vtype=GRB.BINARY, name="covered")

# Budget constraint
model.addConstr(sum(basic[i] * 220000 + ext[i] * 490000 for i in range(n)) <= 1_200_000)

# Coverage logic
for j in range(n):
    model.addConstr(
        covered[j] <= sum(coverage_bas.iloc[j, i] * basic[i] + coverage_ext.iloc[j, i] * ext[i] for i in range(n))
    )

# Constraint: Springfield must be covered by at least one extended antenna
springfield_idx = locations.index("Springfield")
model.addConstr(sum(coverage_ext.iloc[springfield_idx, i] * ext[i] for i in range(n)) >= 1)

# Objective: Maximize covered population
model.setObjective(sum(covered[i] * pop[i] for i in range(n)), GRB.MAXIMIZE)

# Solve model
model.optimize()

# Output results
if model.status == GRB.OPTIMAL:
    total_covered_pop = 0
    print("\nOptimal Antenna Placement:")
    for i in range(n):
        if basic[i].x > 0.5:
            print(f"Basic Antenna at {locations[i]}")
        elif ext[i].x > 0.5:
            print(f"Extended Antenna at {locations[i]}")
    print("\nCovered Regions:")
    for i in range(n):
        if covered[i].x > 0.5:
            print(f"{locations[i]} (Population: {pop[i]})")
            total_covered_pop += pop[i]
    print(f"\nTotal Covered Population: {total_covered_pop}")
    print(f"Coverage Percentage: {100 * total_covered_pop / sum(pop):.2f}%")
else:
    print("No optimal solution found.")


Set parameter Username
Set parameter LicenseID to value 2625267
Academic license - for non-commercial use only - expires 2026-02-19
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 28 rows, 78 columns and 263 nonzeros
Model fingerprint: 0x02b678ff
Variable types: 0 continuous, 78 integer (78 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+05]
  Objective range  [4e+03, 2e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Found heuristic solution: objective 260000.00000
Presolve removed 7 rows and 35 columns
Presolve time: 0.00s
Presolved: 21 rows, 43 columns, 117 nonzeros
Found heuristic solution: objective 300200.00000
Variable types: 0 continuous, 43 integer (43 binary)
Found heuristic solution: objective 377100.00000

Root relaxation: objective 5.202182e+05, 22 iterations, 0.00 seconds (0.00 wor